# GEOG696C Spatiotemporal Data Analysis
## Homework #5
Alex Saunders | Last updated: 30 October 2023

In [1]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt, colors
from matplotlib.collections import LineCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patheffects as pe
from datetime import datetime
import calendar
import scipy
import xarray
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature

In [2]:
# Set the root path
rootPath = Path('C:/Users/alexa/Documents/GitHub/spatiotemporal_data_analysis/00_hw')
dataPath = Path ('C:/Users/alexa/Documents/01_personal/07_phd/05_study/02_courses/2023/1/GEOG696/data')

## 1. Perform a field correlation between winter (DJF) SOI time series and the global DJF 700mb heights and calculate associated p-values